In [1]:
# visit http://127.0.0.1:8050/ in your web browser.
# Imports
import torch
from torch.autograd.functional import jacobian
import torch.nn.functional as F
import cv2 

import matplotlib.pyplot as plt

import sys
# Add the upper directory to the path
sys.path.append("../../models/")
from CustomCNNVessel import CustomResNet
sys.path.append("../../data/")
from VessMapDatasetLoader import vess_map_dataloader

import dash
from dash import html, dcc, Dash, html, dcc, Input, Output, callback, State
from dash.dependencies import Input, Output, MATCH
import plotly.express as px
import json

torch.cuda.empty_cache()
device = torch.device("cuda")

In [2]:
""" # Old Model
model_custom_net = CustomResNet(num_classes=2).to(device)
# Load the weights
model_custom_net.load_state_dict(torch.load(f"../../models/vess_map_regularized_none_200.pth"))
model_custom_net = model_custom_net.eval() """

' # Old Model\nmodel_custom_net = CustomResNet(num_classes=2).to(device)\n# Load the weights\nmodel_custom_net.load_state_dict(torch.load(f"../../models/vess_map_regularized_none_200.pth"))\nmodel_custom_net = model_custom_net.eval() '

In [3]:
# Augmented train model
sys.path.append("/home/fonta42/Desktop/interpretacao-redes-neurais/models/augmented_vessels_trained_models/inferencia_baseline")
import torchtrainer
import torch
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

# instanciando o modelo
model_layers = (1, 1, 1)
model_channels = (64, 64, 64)
model_augment = torchtrainer.models.resunet.ResUNetV2((8,), (1,), (64,)).to('cuda')
# carregando o state_dict
baseline_state_dict = torch.load('/home/fonta42/Desktop/interpretacao-redes-neurais/models/augmented_vessels_trained_models/inferencia_baseline/unetv2_baseline/checkpoint_best.pth')['model']
model_augment.load_state_dict(baseline_state_dict)
model_augment.eval();

In [4]:
import torchseg

pretrained_unet_restnet_model = torchseg.Unet(
    encoder_name="resnet18",
    encoder_weights=True,
    in_channels=1,
    classes=2,
).to('cuda')

pretrained_unet_restnet_model.load_state_dict(torch.load("/home/fonta42/Desktop/interpretacao-redes-neurais/models/torchseg/pretrained_unet_resnet.pt"))

<All keys matched successfully>

In [5]:
unet_restnet_model = torchseg.Unet(
    encoder_name="resnet18",
    encoder_weights=False,
    in_channels=1,
    classes=2,
).to('cuda')

unet_restnet_model.load_state_dict(torch.load("/home/fonta42/Desktop/interpretacao-redes-neurais/models/torchseg/unet_resnet.pt"))

<All keys matched successfully>

In [6]:
# Original Images
import os
from PIL import Image
import numpy as np

def load_images_from_directory(directory_name):
    # Get the list of image file names in sorted order
    image_files = sorted(os.listdir(directory_name))

    # Load and store the images in a list
    images = []
    for file_name in image_files:
        if file_name.endswith('.png'):
            img_path = os.path.join(directory_name, file_name)
            img = Image.open(img_path)
            img_array = np.array(img)
            images.append(img_array)

    return images

# Load images from both directories
original_images = load_images_from_directory('../cropped_images')

# Convert the lists to arrays if needed
original_images = torch.tensor(np.array(original_images) / 255.0, dtype=torch.float).to('cuda')
original_images.shape

torch.Size([100, 64, 64])

In [7]:
# Augmented Vessels

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def load_and_plot_images(directory, plot, image_names = []):
    # List to hold images as numpy arrays
    image_list = []
    
    # Loop over all files in the directory
    for file_name in os.listdir(directory):
        if file_name.endswith('.tiff'):
            # Full path to the image
            file_path = os.path.join(directory, file_name)
            image_names.append(file_path)
            
    image_names.sort()
    
    for file_path in image_names: 
        # Open the image and convert to a numpy array
        image = Image.open(file_path)
        image_np = np.array(image)
        image_list.append(image_np)
        
        if plot:
            # Plotting the image
            plt.figure(figsize=(6, 6))
            plt.imshow(image_np, cmap='gray')
            plt.title(file_name)
            plt.axis('off')
            plt.show()
    
    return image_list, image_names

# Usage
directory = '/home/fonta42/Desktop/interpretacao-redes-neurais/data/augmented_vessels'
original_images, image_names = load_and_plot_images(directory, plot = False)


# Convert the lists to arrays if needed
original_images = torch.tensor(np.array(original_images) / 255.0, dtype=torch.float).to('cuda')
original_images.shape
# TODO: run for all images

torch.Size([11, 128, 128])

In [8]:
# Original images
""" fulfillment_images = []
for i in range(100):
  fulfillment_images.append(np.load(f'../fulfillment_images/image_{i}.npy'))

fulfillment_images = np.array(fulfillment_images)

fulfillment_images.shape """

" fulfillment_images = []\nfor i in range(100):\n  fulfillment_images.append(np.load(f'../fulfillment_images/image_{i}.npy'))\n\nfulfillment_images = np.array(fulfillment_images)\n\nfulfillment_images.shape "

In [9]:
# fullfilment_images_augmented
""" 
fulfillment_images = []

for i in range(11):
  if i == 9:
        i = '09'
  
  img = np.load(f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/fullfilment_images_augmented/image_{i}.npy')
  
  fulfillment_images.append(np.clip(img, 0, 0.00001))

fulfillment_images = np.array(fulfillment_images)

fulfillment_images.shape """

" \nfulfillment_images = []\n\nfor i in range(11):\n  if i == 9:\n        i = '09'\n  \n  img = np.load(f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/fullfilment_images_augmented/image_{i}.npy')\n  \n  fulfillment_images.append(np.clip(img, 0, 0.00001))\n\nfulfillment_images = np.array(fulfillment_images)\n\nfulfillment_images.shape "

In [10]:
# Fullfilment images
augmented_trained_fulfillment_images = []
pre_trained_fulfillment_images = []
non_trained_fulfillment_images = []


for i in range(11):
    augmented_img = np.load(f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/augmented_vessels_trained_model/image_{i}.npy')
    augmented_trained_fulfillment_images.append(np.clip(augmented_img, 0, 0.00001))
    
    pre_img = np.load(f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/pre_trained_model/image_{i}.npy')
    pre_trained_fulfillment_images.append(np.clip(augmented_img, 0, 0.00001))
    
    non_img = np.load(f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/non_trained_model/image_{i}.npy')
    non_trained_fulfillment_images.append(np.clip(augmented_img, 0, 0.00001))

augmented_trained_fulfillment_images = np.array(augmented_trained_fulfillment_images)
pre_trained_fulfillment_images = np.array(pre_trained_fulfillment_images)
non_trained_fulfillment_images = np.array(non_trained_fulfillment_images)

print(augmented_trained_fulfillment_images.shape)
print(pre_trained_fulfillment_images.shape)
print(non_trained_fulfillment_images.shape)

(11, 128, 128)
(11, 128, 128)
(11, 128, 128)


In [11]:
""" gradient_path = f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/non_trained_model'
fulfillment_images = non_trained_fulfillment_images
model = unet_restnet_model """

""" gradient_path = f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/pre_trained_model'
fulfillment_images = pre_trained_fulfillment_images
model = pretrained_unet_restnet_model """


gradient_path = f'/home/fonta42/Desktop/interpretacao-redes-neurais/experiments/torchseg_gradients/augmented_vessels_trained_model'
fulfillment_images = augmented_trained_fulfillment_images
model = model_augment

In [12]:
def plot_gradients_with_bounding_box(gradient, model_name, threshold=0.01):
    gradient = gradient.squeeze()
    mask = np.abs(gradient) > threshold
    non_zero_coords = np.nonzero(mask)
    
    if len(non_zero_coords[0]) > 0:
        y_min, y_max = non_zero_coords[0].min(), non_zero_coords[0].max()
        x_min, x_max = non_zero_coords[1].min(), non_zero_coords[1].max()
        num_pixels_above_threshold = np.sum(mask)
        bounding_box_area = (y_max - y_min + 1) * (x_max - x_min + 1)
        fulfillment = num_pixels_above_threshold / bounding_box_area

        # Create the figure using Plotly Express
        fig = px.imshow(
            gradient[y_min:y_max+1, x_min:x_max+1],
            title=f'Gradient Analysis for {model_name}',
            labels={'x': 'x-axis', 'y': 'y-axis'},
            color_continuous_scale=px.colors.diverging.RdYlGn,
            range_color=[-np.abs(gradient).max(), np.abs(gradient).max()],
        )

        fig.update_layout(
            annotations=[{
                'text': f"Pixels: {num_pixels_above_threshold}<br>"
                        f"X(min, max): ({x_min}, {x_max})<br>"
                        f"Y(min, max): ({y_min}, {y_max})<br>"
                        f"Area: {bounding_box_area}<br>"
                        f"Fulfillment: {fulfillment:.2f}<br>"
                        f"Threshold: {threshold:.6f}<br>",
                        
                'showarrow': False,
                'xref': 'paper',
                'yref': 'paper',
                'x': 0, 'y': 1,
                'xanchor': 'left', 'yanchor': 'top',
                'font': {'size': 12, 'color': 'black'},
                'bgcolor': 'white',
                'opacity': 0.7
            }],
            xaxis={'visible': False},
            yaxis={'visible': False},
        )
        return fig
    else:
        return px.imshow(torch.zeros(128, 128), color_continuous_scale=px.colors.sequential.gray)

In [13]:
app = Dash(__name__)

app.layout = html.Div(
    style={
        'display': 'flex',
        'flex-direction': 'column',
        'align-items': 'center',
        'height': '95vh',  # Reduced height to account for borders
        'width': '95vw',   # Reduced width to account for borders
        'padding': '10px',  # Add some padding
        'margin': 'auto',  # Center the div
        'box-shadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2)',  # Add a shadow for better visuals
        'background-color': '#f9f9f9',  # Change background color for better contrast
        'border-radius': '15px',  # Add border radius
    },
    children=[
        html.H1("Dynamic Gradient Plots", style={'margin-bottom': '20px'}),
        html.Div(id='hover-data', style={'display': 'none'}),
        html.Div(
            style={
                'display': 'flex',
                'flex-direction': 'row',
                'justify-content': 'space-around',
                'width': '95vw',
                'margin-bottom': '20px',
            },
            children=[
                dcc.Dropdown(
                    id='image-dropdown',
                    options=[{'label': f'Image {i}', 'value': i} for i in range(len(original_images))],
                    value=0,
                    style={
                        'width': '30vw',
                        'margin-right': '10px',
                        'background-color': '#fff',  # White background for better visibility
                        'border-radius': '15px',
                    }
                ),
                dcc.Input(
                    id='threshold-input',
                    type='number',
                    placeholder='Porcentage of max value of gradient as threshold',
                    min=0.001,
                    max=100,
                    style={
                        'width': '30vw',
                        'margin-right': '10px',
                        'padding': '5px',  # Add some padding for better appearance
                        'border-radius': '15px',
                    }
                ),
                html.Button(
                    'Update',
                    id='update-button',
                    n_clicks=0,
                    style={
                        'width': '30vw',
                        'padding': '5px',  # Add some padding for better appearance
                        'background-color': '#007bff',  # Bootstrap primary color
                        'color': '#fff',  # White text color
                        'border': 'none',  # Remove default border
                        'cursor': 'pointer',  # Change cursor to pointer on hover
                        'border-radius': '15px',
                    }
                )
            ]
        ),
        html.Div(
            style={
                'display': 'flex',
                'flex-direction': 'row',
                'width': '95vw',  # Use 100% of the width
                'height': '95vh',  # Adjust the height as needed
            },
            children=[
                html.Div(
                    style={
                        'display': 'flex',
                        'flex-direction': 'column',
                        'align-items': 'center',
                        'flex': '1',
                        'margin-right': '2%',
                        'border-radius': '15px'
                    },
                    children=[
                        html.H3("Original Image", style={'padding':'10px'}),
                        dcc.Graph(
                            id='image-display',
                            config={'displayModeBar': True},
                            style={'height': '80%', 'width': '100%'}
                        )
                    ]
                ),
                html.Div(
                    style={
                        'display': 'flex',
                        'flex-direction': 'column',
                        'align-items': 'center',
                        'flex': '1',
                        'margin-right': '2%',
                        'border-radius': '15px'
                    },
                    children=[
                        html.H3("Normalized quantity of pixels above 1% of max value for gradient at (x,y)"),
                        dcc.Graph(
                            id='fulfillment-image-display',
                            config={'displayModeBar': True},
                            style={'height': '80%', 'width': '100%'}
                        )
                    ]
                ),
                html.Div(
                    style={
                        'display': 'flex',
                        'flex-direction': 'column',
                        'align-items': 'center',
                        'flex': '1',
                        'margin-right': '2%',
                        'border-radius': '15px'
                    },
                    children=[
                        html.H3("Model Mask", style={'padding':'10px'}),
                        dcc.Graph(
                            id='model-mask-display',
                            config={'displayModeBar': True},
                            style={'height': '80%', 'width': '100%'}
                        )
                    ]
                ),
                html.Div(
                    style={
                        'display': 'flex',
                        'flex-direction': 'column',
                        'align-items': 'center',
                        'flex': '1',
                        'margin-right': '2%',
                        'border-radius': '15px'
                    },
                    children=[
                        html.H3(id='hover-coordinates', children='Coordinates: (x, y)', style={'padding':'10px'}),
                        dcc.Graph(
                            id='result-image-display',
                            config={'displayModeBar': True},
                            style={'height': '80%', 'width': '100%'}
                        )
                    ]
                ),
                html.Div(
                    style={
                        'display': 'flex',
                        'flex-direction': 'column',
                        'align-items': 'center',
                        'flex': '1',
                        'border-radius': '15px'
                    },
                    children=[
                        html.H3(id='threshold-title', children="Pixels of gradient above Threshold: 0)"),
                        dcc.Graph(
                            id='gradient-display',
                            config={'displayModeBar': True},
                            style={'height': '80%', 'width': '100%'}
                        )
                    ]
                )
            ]
        )
    ]
)

# Callback to update the original image based on the dropdown selection
@app.callback(
    Output('image-display', 'figure'),
    Output('fulfillment-image-display', 'figure'),
    Output('model-mask-display', 'figure'),
    Input('image-dropdown', 'value'),
)
def update_images(selected_index):
    original_image_data = original_images[selected_index].cpu().numpy()
    
    fulfillment_image_data = fulfillment_images[selected_index]
    
    original_fig = px.imshow(original_image_data, color_continuous_scale=px.colors.sequential.gray)
    
    model_mask = model(original_images[selected_index].unsqueeze(0).unsqueeze(0))
    fulfillment_image_data *= model_mask.argmax(dim=1).squeeze(0).detach().cpu().numpy()
    fulfillment_fig = px.imshow(fulfillment_image_data, color_continuous_scale=px.colors.diverging.RdYlGn)
    
    softmax_probs = F.softmax(model_mask, dim=1)
    class_one_probs = softmax_probs[0, 1, :, :].detach().cpu().numpy()

    model_fig = px.imshow(class_one_probs, color_continuous_scale=px.colors.diverging.RdYlGn)
    
    return original_fig, fulfillment_fig,model_fig

# Define callback to update the hover data
@app.callback(
    Output('hover-data', 'children'),
    Input('image-display', 'hoverData'),
    Input('fulfillment-image-display', 'hoverData')
)
def store_hover_data(hover_data_original, hover_data_fulfillment):
    hover_data = hover_data_original or hover_data_fulfillment
    if hover_data:
        return json.dumps({'x': hover_data['points'][0]['x'], 'y': hover_data['points'][0]['y']})
    return "{}"

# Define callback to update the hover coordinates display
@app.callback(
    Output('hover-coordinates', 'children'),
    Input('hover-data', 'children')
)
def update_hover_coordinates(hover_data_json):
    hover_data = json.loads(hover_data_json)
    if hover_data:
        x, y = hover_data['x'], hover_data['y']
        return f'Full Gradient, at Coordinates: ({x}, {y})'
    return 'Full Gradient, Coordinates: (x, y)'

# Define callback to update the result image
@app.callback(
    Output('result-image-display', 'figure'),
    Input('update-button', 'n_clicks'),
    Input('hover-data', 'children'),
    Input('image-dropdown', 'value'),
)
def update_image(n_clicks, hover_data_json, selected_index):
    hover_data = json.loads(hover_data_json)
    if hover_data:
        x, y = hover_data['x'], hover_data['y']
        loaded_gradient = torch.load(f'{gradient_path}/jacobian_gradient_{selected_index}.pt')
        
        max_val = torch.max(torch.abs(loaded_gradient)).cpu().item()
        #max_val = np.max([np.max(gradient) for gradient in gradient])
        fig = px.imshow(loaded_gradient[y,x].to('cpu'), color_continuous_scale=px.colors.diverging.RdYlGn, range_color=[-max_val, max_val])
        return fig
    return px.imshow(torch.zeros(128, 128), color_continuous_scale=px.colors.diverging.RdYlGn)

# Define callback to update the gradient display and the threshold title
@app.callback(
    Output('gradient-display', 'figure'),
    Output('threshold-title', 'children'),
    Input('hover-data', 'children'),
    Input('threshold-input', 'value'),
    Input('image-dropdown', 'value')
)
def update_gradient_display(hover_data_json, threshold, selected_index):
    hover_data = json.loads(hover_data_json)
    if hover_data:
        x, y = hover_data['x'], hover_data['y']
        loaded_gradient = torch.load(f'{gradient_path}/jacobian_gradient_{selected_index}.pt')
        max_val = torch.max(torch.abs(loaded_gradient)).cpu().item()
        
        threshold = threshold if threshold is not None else 0.01
        threshold_val = (threshold / 100) * max_val

        # Return the gradient with the diverging color map
        fig = plot_gradients_with_bounding_box(loaded_gradient[x,y].to('cpu').numpy(),"Model", threshold=threshold_val)
        fig.update_layout(
            title=f"Pixels of gradient above Threshold: {threshold_val:.4f}"
        )
        return fig, f"Pixels of gradient above Threshold: {threshold_val:.4f}"
    return px.imshow(torch.zeros(128, 128), color_continuous_scale=px.colors.diverging.RdYlGn), "Pixels of gradient above Threshold: 0"

if __name__ == '__main__':
    app.run_server(debug=True)

# Análise de Imagens de Vasos Sanguíneos

## Imagem 0:

- Gradiente bastante estável;
- Valores altos para região de falha de vaso, região (25,34);
- Valores altos para borda superior direita (63,1), região que parece ter um vaso escondido, rede até chegou a classificar como vaso;
- Pequena variação na borda inferior centro (38,63), parece ser uma pequena parte de um vaso;
- No geral, gradientes maiores em regiões onde aparenta ter uma descontinuação de um vaso.

## Imagem 1:

- Valores altos nas bordas do vaso, região (27,28);
- Valores altos na região (28,60), parece ser uma descontinuação de um vaso;
- Valores altos na região (5,28), parece ser a ponta de um vaso sanguíneo;
- Valores de gradiente alto na região (63,3), também parece ser a ponta de um vaso;
- Valores altos na região (16,45), parece algo entre a borda de um vaso e uma possível descontinuação do mesmo vaso.

## Imagem 2:

- Valores altos nos ruídos e descontinuação do vaso região entre (12,18) até (28,30);
- No geral, valores altos nas bordas e na transição entre a borda de um vaso sanguíneo e o fundo.

## Imagem 3:

- Valores altos na região (45,33), parece ser algo entre a borda do vaso sanguíneo e uma possível descontinuação;
- Valores altos na região (61,43), no geral, parece ser um comportamento padrão de gradiente com valores altos para regiões onde a probabilidade de ser um vaso fica próximo a 50%, regiões onde a rede não dá uma probabilidade para nenhuma das duas possíveis targets;
- Valores altos na região (2,15), mesma questão da observação acima.

## Imagem 4:

- Valores altos em um ruído com destaque no fundo da imagem, região (39,44), mesma observação de parece ser uma zona de "dúvida" da rede neural;
- Valores altos na região (62,57), mesmo caso das anteriores, região com probabilidades em torno de 50%;
- Região (58,3), (8,1), mesmo comportamento.
